In [1]:
import nltk
import evaluate
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

c:\Users\erwin\miniconda3\envs\stega\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [3]:
# Load the tokenizer, model, and data collator
#MODEL_NAME = "google/flan-t5-base"
#MODEL_NAME = "mistralai/Mistral-7B-v0.1"
#MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
#MODEL_NAME = "Qwen/Qwen2.5-0.5B"
#MODEL_NAME = "Qwen/Qwen3-0.6B"
MODEL_NAME = "HuggingFaceTB/SmolLM2-360M-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
	MODEL_NAME, 
    dtype="auto"
).to(device)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [4]:
prompt = "Write a step by step recipe to make bolognese pasta"

In [5]:
messages = \
[
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)

inputs = tokenizer([text], return_tensors="pt").to(model.device)

print(text)

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
Write a step by step recipe to make bolognese pasta<|im_end|>
<|im_start|>assistant



In [ ]:
outputs = model.generate(**inputs, max_new_tokens=1024, temperature=0.2, top_p=0.9, do_sample=True)


In [7]:
output_ids = outputs[0][len(inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

In [8]:
from IPython.display import display, Markdown, Latex
display(Markdown(content))

1. **Prepare the Bolognese Sauce**: In a large pot, heat 2 tablespoons of olive oil over medium heat. Add 2 minced garlic cloves and sauté until fragrant. Add 1 cup of beef broth, 1 cup of red wine, and 1 cup of beef broth. Bring to a boil, then reduce the heat to low and simmer for 20 minutes.

2. **Cook the Pasta**: In a large pot of boiling salted water, cook the spaghetti according to the package instructions. Drain the pasta and set aside.

3. **Cook the Meatballs**: In a large skillet, heat 1 tablespoon of olive oil over medium heat. Add 1 pound of ground beef and cook until browned. Add 1 cup of tomato sauce and 1 cup of red wine. Simmer for 10 minutes.

4. **Add the Bolognese Sauce and Pasta**: Pour the bolognese sauce over the cooked pasta. Stir well to combine.

5. **Season the Bolognese**: Sprinkle 1/2 teaspoon of salt and 1/4 teaspoon of black pepper over the bolognese sauce.

6. **Garnish the Pasta**: Sprinkle 1/2 cup of grated Parmesan cheese over the top of the bolognese sauce.

7. **Cook the Pasta**: Cover the pot and cook the pasta according to the package instructions.

8. **Serve the Bolognese Pasta**: Ladle the bolognese sauce over the cooked pasta. Garnish with additional Parmesan cheese and a sprinkle of salt.

Enjoy your homemade bolognese pasta!